In [ ]:
#pip install folium

In [ ]:
#!pip install folium
import folium

# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [ ]:
#pip install geopandas

In [ ]:
#!pip install geopandas
import geopandas as gpd
import pandas as pd
import numpy as np
from urllib import request
import json

In [ ]:
url1 = "https://raw.githubusercontent.com/tttdddnet/Python-Jupyter-Geo/88f57d23e1a7fdc283eec9e255fd0a2dc3103e12/geo_ru.json"
with request.urlopen(url1) as f:
    geo= json.load(f)
df1 = gpd.read_file(url1)
df1.rename(columns = {'name':'region'}, inplace = True)

In [ ]:
df1

In [ ]:
seva = df1[df1['region'] == "Sevastopol'"]
seva= seva.drop(['id','Subject_translit','Subject_en'], axis = 1)
seva

In [ ]:
crimea = df1[df1['region'] == 'Crimea']
crimea = crimea.drop(['id','Subject_translit','Subject_en'], axis = 1)
crimea

In [ ]:
url = "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/russia.geojson"
with request.urlopen(url) as f:
    geo= json.load(f)
df = gpd.read_file(url)
df.rename(columns = {'name':'region'}, inplace = True)

In [ ]:
df = df.drop(['cartodb_id', 'created_at','updated_at','name_latin'], axis = 1)
df

In [ ]:
df = pd.concat([df, crimea])
df = pd.concat([df, seva])
df['region'] = df['region'].replace({'Crimea':"Республика Крым"})
df['region'] = df['region'].replace({"Sevastopol'":"г. Севастополь"})
df

In [ ]:
df3 = pd.read_excel("divorce.xls")
df3['region'] = df3['region'].str.strip()
df3['region'] = df3['region'].replace({'Тюменская обл.без данных по Ханты-Мансийскому и Ямало-Ненецкому авт. окр.':'Тюменская область без автономий'})
df3['region'] = df3['region'].replace({'Ханты-Мансийский автономный округ - Югра (Тюменская область)': 'Ханты-Мансийский авт. округ – Югра'})
df3['region'] = df3['region'].replace({'Ямало-Ненецкий автономный округ (Тюменская область)': 'Ямало-Ненецкий автономный округ'})
df3['region'] = df3['region'].replace({'Архангельская обл. без данных по Ненецкому авт. окр.': 'Архангельская область без автономии'})
df3['region'] = df3['region'].replace({"Ненецкий автономный округ (Архангельская область)":"Ненецкий автономный округ"})
df3

In [ ]:
#df['region'].to_excel("outp.xlsx")
reg1 = {"Республика Бурятия":"Бурятия",
"Город Москва столица Российской Федерации город федерального значения":"Москва",
"г. Москва":"Москва",
"Город Санкт-Петербург город федерального значения":"Санкт-Петербург",
"г. Санкт-Петербург":"Санкт-Петербург",
"Республика Тыва":"Тыва",
"Кабардино-Балкарская Республика":"Кабардино-Балкарская республика",
#Чукотский автономный округ
"Республика Адыгея":"Адыгея",
"Республика Татарстан":"Татарстан",
"Республика Марий Эл":"Марий Эл",
"Чувашская Республика":"Чувашия",
"Республика Северная Осетия - Алания":"Северная Осетия - Алания",
"Республика Северная Осетия-Алания":"Северная Осетия - Алания",
"Республика Алтай":"Алтай",
"Республика Дагестан":"Дагестан",
"Кемеровская область - Кузбасс":"Кемеровская область",
"Республика Ингушетия":"Ингушетия",
"Республика Башкортостан": "Башкортостан",
"Удмуртская Республика" :"Удмуртская республика",
"Республика Адыгея (Адыгея)":"Адыгея",
"Карачаево-Черкесская Республика":"Карачаево-Черкесская республика",
"Республика Северная Осетия – Алания":"Северная Осетия - Алания",
"Чеченская Республика":"Чеченская республика",
"Чувашская Республика - Чувашия":"Чувашия",
"Республика Татарстан (Татарстан)":"Татарстан",
"Тюменская область без автономий":"Тюменская область",
"Ханты-Мансийский авт. округ – Югра":"Ханты-Мансийский автономный округ - Югра",
"Архангельская область без автономии":"Архангельская область",
"Город федерального значения Севастополь":"г. Севастополь"
       }
df3['region'] = df3['region'].replace(reg1)
df3[df3['region'] == 'Северная Осетия - Алания']

In [ ]:
# df["lon"] = df["geometry"].centroid.x
# df["lat"] = df["geometry"].centroid.y
#df['value'] = np.random.randint(5,50,36)
#df
df3 = df3.merge(df, on='region')
df3

In [ ]:
russia_map = folium.Map(
    location = [64.6863136, 97.7453061],    # широта и долгота России
    zoom_start = 4
)

#display (russia_map)

In [ ]:
map = folium.Map([68 ,103],     
                        zoom_start=3,
                        width="%100",
                        height="%100")
folium.Choropleth(
    geo_data=df,
    data=df3,
    name = 'Разводы',
    columns=['region', 'k'],   
    legend_name='Коэффициент разводимости на 1000 человек',
    key_on='feature.properties.region',
    fill_color="PuBu",
    fill_opacity=0.8,
    line_opacity=0.5,
    ).add_to(map)
folium.LayerControl().add_to(map)

tooltip = "Click Here For More Info"
# координаты середины канала - 66°25' с. ш. 94°15' в. д
marker = folium.Marker(
    location=[66.25, 91.15],
    popup="<stong><a href='https://t.me/maps_stat'>tg: https://t.me/maps_stat</a><br /><a href='https://vk.com/maps_stat'>vk: https://vk.com/maps_stat</a></stong>",
    icon=folium.DivIcon(html=f"""
      <div style="color:#878787;background:#878787:90px;text-align:center;"><h5><u>@maps_stat</u><h5></div>
    """),
    fill_opacity=0.5
    )
marker.add_to(map)
map

In [ ]:
map.save("divorce.html")

In [ ]:
# про Чукотку статья https://habr.com/ru/post/459902/